# Portugal Energy System Model - PyPSA Analysis

**Group Q** - Data Science for Energy System Modeling (DSESM)

---

## Project Overview

This notebook implements a comprehensive PyPSA-based energy system model for Portugal. We analyze the electricity sector with focus on:
- Renewable energy integration (solar, wind, hydro)
- Network topology and constraints
- Optimal capacity expansion
- Policy scenarios and their impacts


**Team Members:**
- Avinash Varghese
- Sunder Shrestha

## 1. Setup and Configuration

Import required libraries and set up the environment.

In [1]:
# Importing libraries
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Geospatial libraries
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Atlite for renewable resource assessment
import atlite

# Configuration
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully")
print(f"PyPSA version: {pypsa.__version__}")

c:\Users\nashm\anaconda3\envs\dsesm\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ All libraries imported successfully
PyPSA version: 0.33.2


### 1.1 Define Project Paths and Parameters

In [ ]:
# Project directory structure
BASE_DIR = Path.cwd()
DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"
RESULTS_DIR = BASE_DIR / "results"
FIGURES_DIR = BASE_DIR / "figures"

# Create directories if they don't exist
for directory in [DATA_RAW, DATA_PROCESSED, RESULTS_DIR, FIGURES_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Model parameters
COUNTRY = "Portugal"
YEAR = 2025
SNAPSHOT_HOURS = 8760  # Full year hourly resolution
SOLVER = "gurobi"  # Commercial solver

print(f"📁 Project directory: {BASE_DIR}")
print(f"🌍 Country: {COUNTRY}")
print(f"📅 Year: {YEAR}")
print(f"⚡ Solver: {SOLVER}")

---

## 2. Data Collection and Download

Download necessary data for the energy system model.

### 2.1 Geographic Data

Load or download geographic boundaries for Portugal.

In [ ]:
# Download/Load Portugal geographic boundaries
import json

# Define paths
regions_dir = DATA_PROCESSED / "regions"
regions_dir.mkdir(parents=True, exist_ok=True)
portugal_geojson_path = regions_dir / "portugal_boundaries.geojson"

# Check if file already exists
if portugal_geojson_path.exists():
    print(f"📍 Loading existing Portugal boundaries from {portugal_geojson_path}")
    portugal_boundaries = gpd.read_file(portugal_geojson_path)
else:
    print("📍 Creating Portugal geographic boundaries...")
    
    # Create simplified GeoJSON for Portugal mainland
    # Coordinates represent the approximate boundary of mainland Portugal
    portugal_geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "name": "Portugal",
                    "name_long": "Portugal",
                    "iso_a3": "PRT",
                    "continent": "Europe",
                    "region": "Southern Europe"
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [[
                        [-9.5, 42.15], [-9.5, 41.86], [-8.9, 41.86], [-8.5, 41.54],
                        [-7.3, 41.96], [-6.95, 41.93], [-6.19, 41.54], [-6.39, 41.38],
                        [-6.85, 41.11], [-6.86, 40.33], [-7.03, 40.18], [-7.07, 39.71],
                        [-7.50, 39.66], [-7.45, 39.39], [-7.90, 39.15], [-8.98, 38.51],
                        [-9.04, 38.38], [-9.42, 38.73], [-9.29, 38.89], [-9.52, 38.74],
                        [-8.98, 38.41], [-8.79, 37.87], [-8.90, 37.01], [-8.66, 37.02],
                        [-8.67, 37.38], [-8.89, 37.90], [-8.99, 38.51], [-9.09, 38.62],
                        [-9.39, 38.73], [-9.18, 39.03], [-9.03, 39.18], [-8.98, 39.42],
                        [-9.21, 39.63], [-9.16, 40.10], [-8.67, 40.90], [-8.68, 41.04],
                        [-8.78, 41.71], [-8.88, 41.84], [-9.03, 41.88], [-8.98, 42.04],
                        [-9.26, 42.15], [-9.5, 42.15]
                    ]]
                }
            }
        ]
    }
    
    # Save to file
    with open(portugal_geojson_path, 'w') as f:
        json.dump(portugal_geojson, f, indent=2)
    
    # Load the boundaries
    portugal_boundaries = gpd.read_file(portugal_geojson_path)
    print(f"✅ Created and saved Portugal boundaries to {portugal_geojson_path}")

# Display information about the boundaries
print(f"\n📊 Portugal Boundaries Summary:")
print(f"   CRS: {portugal_boundaries.crs}")
print(f"   Geometry type: {portugal_boundaries.geometry.type.values[0]}")
print(f"   Bounds (lon, lat): {portugal_boundaries.total_bounds}")
print(f"   Area: {portugal_boundaries.geometry.area.values[0]:.2f} square degrees")
print(f"\n✅ Portugal geographic boundaries loaded successfully!")

### 2.2 Load Data (Electricity Demand)

Download historical electricity demand data for Portugal.

In [ ]:
# TODO: Download load data
# Sources:
# - ENTSO-E Transparency Platform
# - National grid operator (REN)
# - Historical demand profiles

# Placeholder
print("⚡ Load data download to be implemented")

### 2.3 Generation Capacity Data

Collect existing power plant and capacity data.

In [ ]:
# TODO: Download generation capacity data
# Sources:
# - ENTSO-E power plant database
# - powerplantmatching library
# - National statistics

# Placeholder
print("🏭 Generation capacity data download to be implemented")

### 2.4 Weather Data for Renewable Resources

Use Atlite to download ERA5 weather data for renewable capacity factor calculations.

In [ ]:
# TODO: Download ERA5 weather data using Atlite
# Required variables:
# - Wind speed (for wind power)
# - Solar irradiance (for PV)
# - Temperature
# - Surface pressure

# Placeholder
print("☀️ Weather data download to be implemented")
print("Note: Requires CDS API configuration")

---

## 3. Data Processing and Preparation

Clean and prepare all data for modeling.

### 3.1 Process Geographic Eligibility

Determine eligible areas for renewable energy deployment.

In [ ]:
# TODO: Process eligibility data
# Exclusions:
# - Protected areas
# - Urban areas
# - Water bodies
# - Steep slopes for wind/solar

# Placeholder
print("🗺️ Eligibility processing to be implemented")

### 3.2 Calculate Renewable Capacity Factors

Use Atlite to calculate capacity factors for wind and solar.

In [ ]:
# TODO: Calculate capacity factors using Atlite
# Technologies:
# - Onshore wind
# - Solar PV
# - Offshore wind (optional)

# Placeholder
print("🌬️ Capacity factor calculation to be implemented")

### 3.3 Process Load Profiles

Clean and format electricity demand time series.

In [ ]:
# TODO: Process load data
# Steps:
# - Handle missing values
# - Resample to hourly
# - Validate total consumption

# Placeholder
print("📊 Load profile processing to be implemented")

---

## 4. PyPSA Network Building

Construct the energy system model.

### 4.1 Initialize Network

Create the base PyPSA network object.

In [ ]:
# TODO: Initialize PyPSA network
# Create network with snapshots

# Placeholder
print("🔌 Network initialization to be implemented")

### 4.2 Add Buses (Nodes)

Define the network nodes/buses.

In [ ]:
# TODO: Add buses to network
# Options:
# - Single node (copper plate)
# - Regional nodes
# - Detailed network topology

# Placeholder
print("📍 Bus addition to be implemented")

### 4.3 Add Generators

Add generation technologies and capacities.

In [ ]:
# TODO: Add generators to network
# Technologies:
# - Solar PV
# - Onshore wind
# - Hydro (run-of-river and reservoir)
# - Natural gas
# - Coal (if applicable)

# Parameters:
# - p_nom (nominal capacity)
# - p_nom_extendable (allow capacity expansion)
# - marginal_cost
# - capital_cost
# - efficiency
# - carrier (technology type)

# Placeholder
print("⚡ Generator addition to be implemented")

### 4.4 Add Loads

Add electricity demand to buses.

In [ ]:
# TODO: Add loads to network
# Use processed demand time series

# Placeholder
print("📊 Load addition to be implemented")

### 4.5 Add Transmission Lines (Optional)

Add transmission infrastructure if using multi-node model.

In [ ]:
# TODO: Add transmission lines
# Parameters:
# - s_nom (capacity)
# - x (reactance)
# - r (resistance)
# - length

# Placeholder
print("🔌 Transmission line addition to be implemented (if multi-node)")

### 4.6 Add Storage (Optional)

Add battery or other storage technologies.

In [ ]:
# TODO: Add storage units
# Types:
# - Battery storage
# - Pumped hydro storage

# Placeholder
print("🔋 Storage addition to be implemented (optional)")

### 4.7 Verify Network

Check network consistency and display summary.

In [ ]:
# TODO: Verify and display network
# Check:
# - All buses have loads or generators
# - No isolated components
# - Capacity factors properly assigned
# - Cost parameters reasonable

# Placeholder
print("✅ Network verification to be implemented")

---

## 5. Model Optimization

Run the optimization to find optimal dispatch and/or capacity expansion.

### 5.1 Run Optimization

Execute the PyPSA optimization.

In [ ]:
# TODO: Run optimization
# network.optimize(solver_name=SOLVER)

# Placeholder
print("⚡ Optimization execution to be implemented")

### 5.2 Check Optimization Status

Verify that optimization converged successfully.

In [ ]:
# TODO: Check optimization status
# Print objective value and solver status

# Placeholder
print("✅ Optimization status check to be implemented")

---

## 6. Results Analysis and Visualization

Analyze and visualize the optimization results.

### 6.1 System Costs

Calculate and display total system costs.

In [ ]:
# TODO: Calculate system costs
# - Total objective value
# - Breakdown by component (capital, operation, fuel)
# - Cost per MWh

# Placeholder
print("💰 System cost analysis to be implemented")

### 6.2 Generation Mix

Analyze the electricity generation mix.

In [ ]:
# TODO: Analyze generation mix
# - Total generation by technology
# - Capacity factors
# - Annual energy production

# Placeholder
print("⚡ Generation mix analysis to be implemented")

### 6.3 Capacity Expansion Results

Display optimal capacity investments (if capacity expansion was allowed).

In [ ]:
# TODO: Analyze capacity expansion
# - Optimal additions by technology
# - Comparison to existing capacity

# Placeholder
print("🏗️ Capacity expansion analysis to be implemented")

### 6.4 Time Series Visualization

Plot generation and demand over time.

In [ ]:
# TODO: Create time series plots
# - Dispatch stack plot
# - Load vs generation
# - Renewable curtailment (if any)
# - Storage operation

# Placeholder
print("📈 Time series visualization to be implemented")

### 6.5 Emissions Analysis

Calculate CO₂ emissions.

In [ ]:
# TODO: Calculate emissions
# - Total CO₂ emissions
# - Emissions intensity (gCO₂/kWh)
# - Breakdown by fuel type

# Placeholder
print("🌍 Emissions analysis to be implemented")

### 6.6 Geographic Visualization

Create maps showing capacity distribution (if multi-node model).

In [ ]:
# TODO: Create geographic maps
# - Network topology
# - Generator locations and sizes
# - Transmission lines
# - Renewable resource potential

# Placeholder
print("🗺️ Geographic visualization to be implemented")

---

## 7. Scenario Analysis (Optional)

Compare different scenarios or sensitivity analysis.

### 7.1 Define Scenarios

Define alternative scenarios to compare (e.g., different CO₂ constraints, cost assumptions).

In [ ]:
# TODO: Define scenarios
# Examples:
# - Baseline
# - High renewable target
# - CO₂ cap
# - No nuclear/coal
# - Different cost assumptions

# Placeholder
print("🎯 Scenario definition to be implemented (optional)")

### 7.2 Run Scenarios

Execute optimization for each scenario.

In [ ]:
# TODO: Run scenario loop
# For each scenario, optimize and store results

# Placeholder
print("🔄 Scenario execution to be implemented (optional)")

### 7.3 Compare Scenarios

Create comparison plots and tables.

In [ ]:
# TODO: Compare scenario results
# - Cost comparison
# - Generation mix comparison
# - Emissions comparison
# - Capacity comparison

# Placeholder
print("📊 Scenario comparison to be implemented (optional)")

---

## 8. Conclusions and Summary

Summarize key findings and insights.

### Key Findings

TODO: Summarize main results:
- Optimal generation mix
- Total system costs
- CO₂ emissions levels
- Renewable energy share
- Key bottlenecks or constraints
- Policy recommendations

### Limitations and Future Work

TODO: Discuss:
- Model assumptions and their impact
- Data limitations
- Suggested improvements
- Additional scenarios to explore

---

## 9. Export Results

Save results and figures for reporting.

In [ ]:
# TODO: Save results
# - Export network to NetCDF
# - Save key metrics to CSV
# - Export figures

# Placeholder
print("💾 Results export to be implemented")

In [ ]:
# TODO: Create geographic maps
# - Network topology
# - Generator locations and sizes
# - Transmission lines
# - Renewable resource potential

# Placeholder
print("🗺️ Geographic visualization to be implemented")

### 6.6 Geographic Visualization

Create maps showing capacity distribution (if multi-node model).

In [ ]:
# TODO: Calculate emissions
# - Total CO₂ emissions
# - Emissions intensity (gCO₂/kWh)
# - Breakdown by fuel type

# Placeholder
print("🌍 Emissions analysis to be implemented")

### 6.5 Emissions Analysis

Calculate CO₂ emissions.

In [ ]:
# TODO: Create time series plots
# - Dispatch stack plot
# - Load vs generation
# - Renewable curtailment (if any)
# - Storage operation

# Placeholder
print("📈 Time series visualization to be implemented")

### 6.4 Time Series Visualization

Plot generation and demand over time.

In [ ]:
# TODO: Analyze capacity expansion
# - Optimal additions by technology
# - Comparison to existing capacity

# Placeholder
print("🏗️ Capacity expansion analysis to be implemented")

### 6.3 Capacity Expansion Results

Display optimal capacity investments (if capacity expansion was allowed).

In [ ]:
# TODO: Analyze generation mix
# - Total generation by technology
# - Capacity factors
# - Annual energy production

# Placeholder
print("⚡ Generation mix analysis to be implemented")

### 6.2 Generation Mix

Analyze the electricity generation mix.

In [ ]:
# TODO: Calculate system costs
# - Total objective value
# - Breakdown by component (capital, operation, fuel)
# - Cost per MWh

# Placeholder
print("💰 System cost analysis to be implemented")

### 6.1 System Costs

Calculate and display total system costs.

---

## 6. Results Analysis and Visualization

Analyze and visualize the optimization results.

In [ ]:
# TODO: Check optimization status
# Print objective value and solver status

# Placeholder
print("✅ Optimization status check to be implemented")

### 5.2 Check Optimization Status

Verify that optimization converged successfully.

In [ ]:
# TODO: Run optimization
# network.optimize(solver_name=SOLVER)

# Placeholder
print("⚡ Optimization execution to be implemented")

### 5.1 Run Optimization

Execute the PyPSA optimization.

---

## 5. Model Optimization

Run the optimization to find optimal dispatch and/or capacity expansion.

In [ ]:
# TODO: Verify and display network
# Check:
# - All buses have loads or generators
# - No isolated components
# - Capacity factors properly assigned
# - Cost parameters reasonable

# Placeholder
print("✅ Network verification to be implemented")

### 4.7 Verify Network

Check network consistency and display summary.

In [ ]:
# TODO: Add storage units
# Types:
# - Battery storage
# - Pumped hydro storage

# Placeholder
print("🔋 Storage addition to be implemented (optional)")

### 4.6 Add Storage (Optional)

Add battery or other storage technologies.

In [ ]:
# TODO: Add transmission lines
# Parameters:
# - s_nom (capacity)
# - x (reactance)
# - r (resistance)
# - length

# Placeholder
print("🔌 Transmission line addition to be implemented (if multi-node)")

### 4.5 Add Transmission Lines (Optional)

Add transmission infrastructure if using multi-node model.

In [ ]:
# TODO: Add loads to network
# Use processed demand time series

# Placeholder
print("📊 Load addition to be implemented")

### 4.4 Add Loads

Add electricity demand to buses.

In [ ]:
# TODO: Add generators to network
# Technologies:
# - Solar PV
# - Onshore wind
# - Hydro (run-of-river and reservoir)
# - Natural gas
# - Coal (if applicable)

# Parameters:
# - p_nom (nominal capacity)
# - p_nom_extendable (allow capacity expansion)
# - marginal_cost
# - capital_cost
# - efficiency
# - carrier (technology type)

# Placeholder
print("⚡ Generator addition to be implemented")

### 4.3 Add Generators

Add generation technologies and capacities.

In [ ]:
# TODO: Add buses to network
# Options:
# - Single node (copper plate)
# - Regional nodes
# - Detailed network topology

# Placeholder
print("📍 Bus addition to be implemented")

### 4.2 Add Buses (Nodes)

Define the network nodes/buses.

In [ ]:
# TODO: Initialize PyPSA network
# Create network with snapshots

# Placeholder
print("🔌 Network initialization to be implemented")

### 4.1 Initialize Network

Create the base PyPSA network object.

---

## 4. PyPSA Network Building

Construct the energy system model.

In [ ]:
# TODO: Process load data
# Steps:
# - Handle missing values
# - Resample to hourly
# - Validate total consumption

# Placeholder
print("📊 Load profile processing to be implemented")

### 3.3 Process Load Profiles

Clean and format electricity demand time series.

In [ ]:
# TODO: Calculate capacity factors using Atlite
# Technologies:
# - Onshore wind
# - Solar PV
# - Offshore wind (optional)

# Placeholder
print("🌬️ Capacity factor calculation to be implemented")

### 3.2 Calculate Renewable Capacity Factors

Use Atlite to calculate capacity factors for wind and solar.

In [ ]:
# TODO: Process eligibility data
# Exclusions:
# - Protected areas
# - Urban areas
# - Water bodies
# - Steep slopes for wind/solar

# Placeholder
print("🗺️ Eligibility processing to be implemented")

### 3.1 Process Geographic Eligibility

Determine eligible areas for renewable energy deployment.

---

## 3. Data Processing and Preparation

Clean and prepare all data for modeling.

In [ ]:
# TODO: Download ERA5 weather data using Atlite
# Required variables:
# - Wind speed (for wind power)
# - Solar irradiance (for PV)
# - Temperature
# - Surface pressure

# Placeholder
print("☀️ Weather data download to be implemented")
print("Note: Requires CDS API configuration")

### 2.4 Weather Data for Renewable Resources

Use Atlite to download ERA5 weather data for renewable capacity factor calculations.

In [ ]:
# TODO: Download generation capacity data
# Sources:
# - ENTSO-E power plant database
# - powerplantmatching library
# - National statistics

# Placeholder
print("🏭 Generation capacity data download to be implemented")

### 2.3 Generation Capacity Data

Collect existing power plant and capacity data.

In [ ]:
# TODO: Download load data
# Sources:
# - ENTSO-E Transparency Platform
# - National grid operator (REN)
# - Historical demand profiles

# Placeholder
print("⚡ Load data download to be implemented")

### 2.2 Load Data (Electricity Demand)

Download historical electricity demand data for Portugal.

In [ ]:
# TODO: Download Portugal geographic boundaries
# Options:
# 1. Use Natural Earth data
# 2. Use GADM administrative boundaries
# 3. Custom shapefile

# Placeholder for geographic data loading
print("📍 Geographic data download/loading to be implemented")

### 2.1 Geographic Data

Load or download geographic boundaries for Portugal.

---

## 2. Data Collection and Download

Download necessary data for the energy system model.

In [ ]:
# Project directory structure
BASE_DIR = Path.cwd()
DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"
RESULTS_DIR = BASE_DIR / "results"
FIGURES_DIR = BASE_DIR / "figures"

# Create directories if they don't exist
for directory in [DATA_RAW, DATA_PROCESSED, RESULTS_DIR, FIGURES_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Model parameters
COUNTRY = "Portugal"
YEAR = 2023
SNAPSHOT_HOURS = 8760  # Full year hourly resolution
SOLVER = "highs"  # Open-source solver

print(f"📁 Project directory: {BASE_DIR}")
print(f"🌍 Country: {COUNTRY}")
print(f"📅 Year: {YEAR}")
print(f"⚡ Solver: {SOLVER}")

### 1.1 Define Project Paths and Parameters

In [ ]:
# Core libraries
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Geospatial libraries
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Atlite for renewable resource assessment
import atlite

# Configuration
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully")
print(f"PyPSA version: {pypsa.__version__}")

## 1. Setup and Configuration

Import required libraries and set up the environment.

# Portugal Energy System Model - PyPSA Analysis

**Group Q** - Data Science for Energy System Modeling (DSESM)

---

## Project Overview

This notebook implements a comprehensive PyPSA-based energy system model for Portugal. We analyze the electricity sector with focus on:
- Renewable energy integration (solar, wind, hydro)
- Network topology and constraints
- Optimal capacity expansion
- Policy scenarios and their impacts

**Date:** January 27, 2026

**Team Members:**
- Member 1
- Member 2
- Member 3
- Member 4

---

## 7. Scenario Analysis (Optional)

Compare different scenarios or sensitivity analysis.

### 7.1 Define Scenarios

Define alternative scenarios to compare (e.g., different CO₂ constraints, cost assumptions).

In [ ]:
# TODO: Define scenarios
# Examples:
# - Baseline
# - High renewable target
# - CO₂ cap
# - No nuclear/coal
# - Different cost assumptions

# Placeholder
print("🎯 Scenario definition to be implemented (optional)")

### 7.2 Run Scenarios

Execute optimization for each scenario.

In [ ]:
# TODO: Run scenario loop
# For each scenario, optimize and store results

# Placeholder
print("🔄 Scenario execution to be implemented (optional)")

### 7.3 Compare Scenarios

Create comparison plots and tables.

In [ ]:
# TODO: Compare scenario results
# - Cost comparison
# - Generation mix comparison
# - Emissions comparison
# - Capacity comparison

# Placeholder
print("📊 Scenario comparison to be implemented (optional)")

---

## 8. Conclusions and Summary

Summarize key findings and insights.

### Key Findings

TODO: Summarize main results:
- Optimal generation mix
- Total system costs
- CO₂ emissions levels
- Renewable energy share
- Key bottlenecks or constraints
- Policy recommendations

### Limitations and Future Work

TODO: Discuss:
- Model assumptions and their impact
- Data limitations
- Suggested improvements
- Additional scenarios to explore

---

## 9. Export Results

Save results and figures for reporting.

In [ ]:
# TODO: Save results
# - Export network to NetCDF
# - Save key metrics to CSV
# - Export figures

# Placeholder
print("💾 Results export to be implemented")